In [ ]:
%load_ext autoreload
%autoreload 2

Description:

Ce notebook est utile pour afficher les courbes de tendance de SoH par modèle (pour l'instant tesla) et on peut même aller plus loin avec les courbe de tendance de SoH pour les versions (ou tesla code).

Il y'a la possibilité d'ajouter les intervals de confiance de ces courbes. 

In [ ]:
import pandas as pd
from core.sql_utils import *
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt

from transform.insights_results.trendlines import *


In [ ]:
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT *  FROM  vehicle v	
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            join vehicle_data vd 
            on vd.vehicle_id  = v.id
            join oem o 
            on o.id = vm.oem_id
            join battery b 
            on b.id = vm.battery_id
            where o.id in ('5b89ebf7-8774-41ec-97f8-7dd4e8a2fd0f');"""), con)



dbeaver_df.head()
dbeaver_df['timestamp'] = pd.to_datetime(dbeaver_df['timestamp'])
dbeaver_df.sort_values("timestamp", inplace=True)

In [ ]:
# on drop MTY13/MT336/MT337
dbeaver_filtered = dbeaver_df[~dbeaver_df['version'].isin(['MT336','MT337'])][['version', 'soh', 'odometer', 'model_name', 'type', 'vin']].copy().dropna()


In [ ]:
px.scatter(dbeaver_filtered[['version', 'soh', 'odometer']],  x='odometer', y='soh', color="version")

## Graphs des tendances 

In [ ]:
trendline, trendline_max, trendline_min = get_trendlines(dbeaver_filtered)

In [ ]:
def trendline_apply(x, f):
    return eval(f)
    
    

In [ ]:
trendline_apply(dbeaver_filtered['odometer'].sort_values(), trendline['trendline'])

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=dbeaver_filtered['odometer'],
    y=dbeaver_filtered['soh'],
    mode='markers',
    marker_color='rgba(50, 182, 193, .9)',
    name='SoH compute'

    ))


fig.add_trace(go.Scatter(
    x=dbeaver_filtered['odometer'].sort_values(),
    y=trendline_apply(dbeaver_filtered['odometer'].sort_values(), trendline['trendline']),
    mode='lines',
    line=dict(color='red'),
    name='Fit'
))


fig.add_trace(go.Scatter(
    x=dbeaver_filtered['odometer'].sort_values(),
    y=trendline_apply(dbeaver_filtered['odometer'].sort_values(), trendline_max['trendline']),
    mode='lines',
    line=dict(color='green'),
    name='upper'
))


fig.add_trace(go.Scatter(
    x=dbeaver_filtered['odometer'].sort_values(),
    y=trendline_apply(dbeaver_filtered['odometer'].sort_values(), trendline_min['trendline']),
    mode='lines',
    line=dict(color='green'),
    name='lower'
))


fig.update_layout(
    width=1000,
    height=600,
    xaxis_title='x',
    yaxis_title='y',
    legend_title='Légende',
    template='plotly_white'
)

fig.show()
